In [29]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF = 1
max_attempts = 2
TIMEOUT_SECONDS = 180  
output_dir="/tmp/llm"
model1 = "o3-mini"
model2 = "qwen3:30b-a3b"

In [30]:
import requests
import json
import duckdb
import pathlib
import time
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os
import threading
import pandas as pd


# Generate Data

In [31]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Run Tests

In [32]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [33]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        con.sql("SET enable_progress_bar_print = false")
        con.sql("SET progress_bar_time = 0")

        result_container = {"result": None, "error": None}

        def query_thread():
            try:
                result_container["result"] = con.execute(current_query).fetchdf()
            except duckdb.InterruptException:
                result_container["error"] = f"Query interrupted after timeout of {TIMEOUT_SECONDS} seconds."
            except Exception as e:
                result_container["error"] = str(e)

        thread = threading.Thread(target=query_thread)
        thread.start()

        start_time = time.time()
        while thread.is_alive():
            elapsed = time.time() - start_time
            if elapsed > TIMEOUT_SECONDS:
                con.interrupt()
                thread.join()
                return f"Query execution timed out after {TIMEOUT_SECONDS} seconds.", attempt, "query runs forever"
            time.sleep(0.1)

        if result_container["error"]:
            error_message = result_container["error"].lower()
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {result_container['error']}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {result_container['error']}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {result_container['error']}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {result_container['error']}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1
        else:
            return result_container["result"], attempt, current_query

    return "Unexpected error or loop termination", attempt, current_query


In [34]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })

    os.makedirs(output_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
            f.flush()
            os.fsync(f.fileno())
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [35]:
def display_side_by_side(nbr, model1, model2):
    print(f"question {nbr} : " + duckdb.sql(f" select question from results_filtered where nbr = {nbr}  ").fetchone()[0])
    try:
        df1 = con.sql(duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model1}' """).fetchone()[0]).df()        
        df2 = con.sql(duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model2}' """).fetchone()[0]).df() 
        side_by_side = pd.concat([df1, df2], axis=1, keys=[model1, model2])
        display(side_by_side)
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

In [36]:
# get the questions from GitHub
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Successfully retrieved questions:


In [37]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

CPU times: total: 0 ns
Wall time: 0 ns


In [38]:
%%time
#ask_question(questions,model2,"ollama")

CPU times: total: 0 ns
Wall time: 0 ns


# Check Results

In [39]:
duckdb.sql(f""" select *,cardinality(result[1]) as nbr_columns, [result_count , nbr_columns] as result_shape, hash(result) as output from '{output_dir}/*.json' where model in ('{model1}','{model2}') """).to_view("results")
try:
 duckdb.sql(f""" install excel ; load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")
# check number of rows and column returned by each model, Null means SQL error
duckdb.sql(f""" 
           with zzzz as (select nbr,question,model,output , timestamp from results )
           pivot( select nbr,model,count(distinct(output)) as result_shape , count(distinct(timestamp)) as nbr_runs from zzzz group by all) 
           on model using min(result_shape) as resultsets ,min(nbr_runs) as nbr_runs order by nbr
        """).show(max_width=120)

Error exporting to Excel: IO Error: Could not move file: The process cannot access the file because it is being used by another process.

close the excel file or tey again.
┌───────┬────────────────────┬──────────────────┬──────────────────────────┬────────────────────────┐
│  nbr  │ o3-mini_resultsets │ o3-mini_nbr_runs │ qwen3:30b-a3b_resultsets │ qwen3:30b-a3b_nbr_runs │
│ int64 │       int64        │      int64       │          int64           │         int64          │
├───────┼────────────────────┼──────────────────┼──────────────────────────┼────────────────────────┤
│     1 │                  3 │                6 │                        1 │                      5 │
│     2 │                  2 │                6 │                        1 │                      5 │
│     3 │                  2 │                6 │                        1 │                      5 │
│     4 │                  3 │                6 │                        1 │                      5 │
│     5 │  

In [40]:
duckdb.sql(f""" 
        create or replace temp table results_filtered as
           from results  where model = '{model1}' and timestamp = (select max(timestamp) from results where model = '{model1}') 
           union all 
           from results  where model = '{model2}' and timestamp = (select max(timestamp) from results where model = '{model2}')
""")
# execusion time in seconds
duckdb.sql(f""" 
           pivot( select nbr as question ,model,result_shape , attempts from results_filtered ) 
           on model using min(result_shape) as result,  min(attempts) as attempts order by question
        """).show(max_width=130)

┌──────────┬────────────────┬──────────────────┬──────────────────────┬────────────────────────┐
│ question │ o3-mini_result │ o3-mini_attempts │ qwen3:30b-a3b_result │ qwen3:30b-a3b_attempts │
│  int64   │    int128[]    │      int64       │       int128[]       │         int64          │
├──────────┼────────────────┼──────────────────┼──────────────────────┼────────────────────────┤
│        1 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        2 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        3 │ [1, 1]         │                1 │ [1, 1]               │                      1 │
│        4 │ [1, 1]         │                1 │ [12, 1]              │                      1 │
│        5 │ [11, 1]        │                1 │ [11, 1]              │                      1 │
│        6 │ [6, 2]         │                1 │ [6, 2]               │                      1 │
│        7 │ [1, 2]         │ 

In [41]:
different_results = duckdb.sql(f"""
    SELECT 
        distinct(nbr) 
    FROM results_filtered 
    WHERE nbr in
    (
    select nbr from results_filtered 
    group by all
    having count(distinct(output)) >  1 
    )
    order by nbr
""").df()['nbr'].tolist()
print(f"Different results for {len(different_results)} questions: {different_results}")
good_results = list(set(list(range(1, 21))) - set(different_results))
print(f"Exact resultsets size for {len(good_results)} questions: {good_results}")


Different results for 20 questions: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Exact resultsets size for 0 questions: []


In [42]:
for nbr in list(range(1, 21)):
    display_side_by_side(nbr, model1, model2)

question 1 : What is the overall total sales revenue?


,o3-mini,qwen3:30b-a3b
,total_sales,total_sales
0,5.138666e+09,5.138666e+09


question 2 : What is the total number of items sold across all transactions?


,o3-mini,qwen3:30b-a3b
,total_quantity,total_quantity
0,138963631.0,138963631.0


question 3 : What is the total monetary value of all returned items?


...

question 18 : Which item brand has shown the largest decrease in its return rate when comparing the average rate of 2001 to 2002, specifically for stores located in the 'TN' state? (Order by the decrease in return rate descending, and then by brand name alphabetically for ties).


o3-mini                                  qwen3:30b-a3b  \
                item_brand decrease_in_return_rate                i_brand   
0      exportiunivamalg #9               20.812102    exportiunivamalg #9   
1         corpunivamalg #3               15.805714       corpunivamalg #3   
2       amalgamalgamalg #5               15.586534     amalgamalgamalg #5   
3         maxiunivamalg #9               15.579876       maxiunivamalg #9   
4    edu packunivamalg #17               15.022974  edu packunivamalg #17   
..                     ...                     ...                    ...   
460      corpamalgamalg #8               -9.763783   scholaramalgamalg #7   
461   scholaramalgamalg #7              -10.325430      amalgunivamalg #6   
462      amalgunivamalg #6              -16.130969   scholarunivamalg #12   
463   scholarunivamalg #12              -40.956686    exportiunivamalg #6   
464    exportiunivamalg #6              -52.860589                    NaN   

                                                                             \
    total_sales_2001 total_sales_2002 total_returns_2001 total_returns_2002   
0          127211.17        379176.84           33625.08           21311.21   
1          116231.48        252400.06           26437.65           17516.50   
2          101515.55        235062.37           16983.12            2686.86   
3           81160.84        171484.23           14871.86            4705.63   
4          111966.37        267425.23           19265.41            5839.10   
..               ...              ...                ...                ...   
460         92858.21        110208.06            4734.17           16998.17   
461        843017.40        171898.92           40547.13           35996.89   
462       1167640.44        101510.82           72795.62           47904.08   
463       1848959.98         71645.01           81566.51           41032.58   
464              NaN              NaN                NaN                NaN   

                                                               
    return_rate_2001 return_rate_2002 decrease_in_return_rate  
0          26.432490         5.620388               20.812102  
1          22.745688         6.939975               15.805714  
2          16.729575         1.143041               15.586534  
3          18.323936         2.744060               15.579876  
4          17.206425         2.183451               15.022974  
..               ...              ...                     ...  
460         5.098278        15.423709              -10.325430  
461         4.809762        20.940731              -16.130969  
462         6.234421        47.191107              -40.956686  
463         4.411481        57.272070              -52.860589  
464              NaN              NaN                     NaN  

[465 rows x 10 columns]

question 19 : For each item class, compare the average net sales value per sales transaction between preferred and non-preferred customers. Order by average net sales descending, and then by item class name alphabetically for ties.


o3-mini                         qwen3:30b-a3b                     \
    item_class preferred avg_net_sales    item_class preferred_customer   
0    wallpaper      None   2078.935543        accent                  Y   
1        decor      None   2071.978736        accent                  N   
2       stereo      None   2049.735175        accent               None   
3      fishing      None   2049.724199   accessories                  N   
4      science      None   2028.626098   accessories               None   
..         ...       ...           ...           ...                ...   
295     sports      None   1684.019397  womens watch                  N   
296    camping      None   1681.420920  womens watch                  Y   
297     custom         Y   1680.005484          None               None   
298     tennis      None   1672.502520          None                  N   
299   flatware      None   1672.412600          None                  Y   

                   
    avg_net_sales  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
..            ...  
295           NaN  
296           NaN  
297           NaN  
298           NaN  
299           NaN  

[300 rows x 6 columns]

question 20 : show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
Error executing query for nbr 20: Parser Error: syntax error at or near "query"
